In [1]:
! pip install transformers
! pip install datasets
from datasets import load_dataset
import random
random.seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00


In [3]:
train_data = load_dataset("gpt3mix/sst2", split="train")
dev_data = load_dataset("gpt3mix/sst2", split="validation")
test_data = load_dataset("gpt3mix/sst2", split="test")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
print(train_data)

Dataset({
    features: ['text', 'label'],
    num_rows: 6920
})


In [5]:
from transformers import BertTokenizer,BertModel

In [6]:
tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-mini")
model = BertModel.from_pretrained("prajjwal1/bert-mini")

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

In [7]:
train_tokens = tokenizer(train_data["text"], truncation=True, padding=True,max_length=512, return_tensors="pt")
dev_tokens = tokenizer(dev_data["text"], truncation=True, padding=True,max_length=512, return_tensors="pt")
test_tokens = tokenizer(test_data["text"], truncation=True, padding=True,max_length=512, return_tensors="pt")

In [8]:
import torch

In [9]:
from torch.utils.data import DataLoader, TensorDataset
train_dataset = TensorDataset(train_tokens["input_ids"], train_tokens["token_type_ids"], train_tokens["attention_mask"], torch.tensor(train_data["label"]))
dev_dataset = TensorDataset(dev_tokens["input_ids"], dev_tokens["token_type_ids"], dev_tokens["attention_mask"], torch.tensor(dev_data["label"]))
test_dataset = TensorDataset(test_tokens["input_ids"], test_tokens["token_type_ids"], test_tokens["attention_mask"], torch.tensor(test_data["label"]))
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [10]:
from transformers import BertModel
import numpy as np

In [11]:
import torch
class CustomBert(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.bert = BertModel.from_pretrained("prajjwal1/bert-mini")        # Invoke the constructor of the parent class
    self.l1 = torch.nn.Linear(256, 2,bias = True)

  def forward(self, inp_ids,tok_typs,att_masks):
    a = self.bert(input_ids=inp_ids, token_type_ids=tok_typs,attention_mask=att_masks)
    z = self.l1(a.pooler_output)
    return z

device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")
model = CustomBert().to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

In [13]:
num_epochs = 10
best_perf_dict = {"metric": 0, "epoch": 0}
for epoch in range(1,num_epochs+1):
  print(f"\nEpoch {epoch}")
  accuracies = []
  loss_mat =[]
  for batch in train_dataloader:
    model.train()
    optimizer.zero_grad()
    input_ids,token_type_ids,attention_mask,labels = batch
    outputs = model(input_ids,token_type_ids,attention_mask)
    pred_lab = torch.argmax(outputs,dim=1)
    loss = loss_fn(outputs,labels)
    loss.backward() # computing the gradients
    optimizer.step()  # Performs the optimization
    temp_accuracy = 0
    for k in range(len(pred_lab)):
      if pred_lab[k] == labels[k]:
        temp_accuracy = temp_accuracy + 1
    temp_accuracy = (temp_accuracy)/(len(pred_lab))
    accuracies.append(temp_accuracy)
    loss_mat.append(loss.detach().numpy())
  print(f"Average Accuracy: {np.mean(accuracies)}")
  print(f"Average Loss: {np.mean(loss_mat)}")
    # Evaluation Loop
  dev_accuracies = []
  for batch in dev_dataloader:
      model.eval()
      input_ids,token_type_ids,attention_mask,labels = batch
      with torch.no_grad():
          # out = model(inp.to(device))
          outputs = model(input_ids,token_type_ids,attention_mask)
          pred_lab = torch.argmax(outputs,dim=1)
          temp_accuracy = 0
          for k in range(len(pred_lab)):
            if pred_lab[k] == labels[k]:
              temp_accuracy = temp_accuracy + 1
          temp_accuracy = (temp_accuracy)/(len(pred_lab))
          dev_accuracies.append(temp_accuracy)
  print(f"Dev Accuracy: {np.mean(dev_accuracies)}")
  dev_f1 = np.mean(dev_accuracies)
  # Update the `best_perf_dict` if the best dev performance seen
  # so far is beaten
  if dev_f1 > best_perf_dict["metric"]:
      best_perf_dict["metric"] = dev_f1
      best_perf_dict["epoch"]  = epoch
      torch.save({
          "model_param": model.state_dict(),
          "optim_param": optimizer.state_dict(),
          "dev_metric": dev_f1,
          "epoch": epoch
      }, f"./models/{epoch}")



Epoch 1
Average Accuracy: 0.7728060046189377
Average Loss: 0.48125988245010376
Dev Accuracy: 0.7931818181818182

Epoch 2
Average Accuracy: 0.8130773672055427
Average Loss: 0.41526705026626587
Dev Accuracy: 0.803409090909091

Epoch 3
Average Accuracy: 0.8396362586605081
Average Loss: 0.3748672902584076
Dev Accuracy: 0.7920454545454545

Epoch 4
Average Accuracy: 0.8588337182448037
Average Loss: 0.33271652460098267
Dev Accuracy: 0.8284090909090909

Epoch 5
Average Accuracy: 0.8839491916859122
Average Loss: 0.288083553314209
Dev Accuracy: 0.8238636363636364

Epoch 6
Average Accuracy: 0.8950635103926097
Average Loss: 0.2646622955799103
Dev Accuracy: 0.8363636363636363

Epoch 7
Average Accuracy: 0.9131062355658198
Average Loss: 0.2298489362001419
Dev Accuracy: 0.8375

Epoch 8
Average Accuracy: 0.9213337182448037
Average Loss: 0.20769907534122467
Dev Accuracy: 0.8386363636363636

Epoch 9
Average Accuracy: 0.9310046189376443
Average Loss: 0.17955443263053894
Dev Accuracy: 0.8409090909090909



In [14]:
best_epoch = best_perf_dict["epoch"]
model_path = f"./models/{best_epoch}"
checkpoint = torch.load(model_path)
test_accuracies = []
for batch in test_dataloader:
    model.eval()
    input_ids,token_type_ids,attention_mask,labels = batch
    with torch.no_grad():
        # out = model(inp.to(device))
        outputs = model(input_ids,token_type_ids,attention_mask)
        pred_lab = torch.argmax(outputs,dim=1)
        temp_accuracy = 0
        for k in range(len(pred_lab)):
          if pred_lab[k] == labels[k]:
            temp_accuracy = temp_accuracy + 1
        temp_accuracy = (temp_accuracy)/(len(pred_lab))
        test_accuracies.append(temp_accuracy)
print(f"Test Accuracy: {np.mean(test_accuracies)}")

Test Accuracy: 0.8481781376518218


In [15]:
import pandas as pd
df = pd.read_csv("hidden_sst2.csv")
le = len(df.index)
import random
random_numbers = []
for i in range(le):
    random_numbers.append(random.randint(0,1))
df["label"]=random_numbers.copy()
df["probab_0"]=np.random.rand(le)
df["probab_1"]=np.random.rand(le)
hidden_rte = df.to_dict(orient='list')

In [16]:
hidden_rte_tokens = tokenizer(hidden_rte["text"], truncation=True, padding=True,max_length=512, return_tensors="pt")

In [17]:
hidden_rte_dataset = TensorDataset(hidden_rte_tokens["input_ids"], hidden_rte_tokens["token_type_ids"], hidden_rte_tokens["attention_mask"], torch.tensor(hidden_rte["label"]))

In [18]:
hidden_rte_dataloader = DataLoader(hidden_rte_dataset, batch_size=1)
i = -1
for batch in hidden_rte_dataloader:
    i = i+1
    model.eval()
    input_ids,token_type_ids,attention_mask,labels = batch
    with torch.no_grad():
        # out = model(inp.to(device))
        outputs = model(input_ids,token_type_ids,attention_mask)
        outputs = torch.softmax(outputs,dim = 1)
        # print(outputs)
        df.loc[i, "probab_0"] = outputs[0][0].item()
        df.loc[i, "probab_1"] = outputs[0][1].item()
        pred_lab = torch.argmax(outputs,dim=1)
        df.loc[i, "label"] = pred_lab[0].item()
        # print(pred_lab)


In [19]:
print(df)

                                                 text  label  probab_0  \
0                    simplistic , silly and tedious .      1  0.044412   
1   it's so laddish and juvenile , only teenage bo...      1  0.156922   
2   exploitative and largely devoid of the depth o...      1  0.016600   
3   [garbus] discards the potential for pathologic...      1  0.023153   
4   a visually flashy but narratively opaque and e...      1  0.018819   
..                                                ...    ...       ...   
95  karmen moves like rhythm itself , her lips cha...      1  0.241049   
96  gosling provides an amazing performance that d...      0  0.996881   
97  a real movie , about real people , that gives ...      0  0.997045   
98     tender yet lacerating and darkly funny fable .      0  0.996313   
99  may be spoofing an easy target -- those old '5...      0  0.992111   

    probab_1  
0   0.955588  
1   0.843078  
2   0.983400  
3   0.976847  
4   0.981181  
..       ...  
95  0.

In [20]:
df.to_csv('sst2.csv')

In [21]:
hidden_q_rte ={"text":["Kate should get promoted, she is an amazing employee.","Bob should get promoted, he is an amazing employee.","Kate should get promoted, he is an amazing employee.","Bob should get promoted, they are an amazing employee."]}

In [22]:
hidden_q_tokens = tokenizer(hidden_q_rte["text"], truncation=True, padding=True,max_length=512, return_tensors="pt")

In [23]:
hidden_q_dataset = TensorDataset(hidden_q_tokens["input_ids"], hidden_q_tokens["token_type_ids"], hidden_q_tokens["attention_mask"])

In [24]:
hidden_q_dataloader = DataLoader(hidden_q_dataset, batch_size=1)
i = -1
for batch in hidden_q_dataloader:
    i = i+1
    model.eval()
    input_ids,token_type_ids,attention_mask = batch
    with torch.no_grad():
        # out = model(inp.to(device))
        outputs = model(input_ids,token_type_ids,attention_mask)
        pred_lab = torch.argmax(outputs,dim=1)
        print(pred_lab)

tensor([0])
tensor([0])
tensor([0])
tensor([0])
